# Test script for the functionality of radioactivedecay

This test script checks the functionality of radioactive decay and checks the output of decay calculations against baseline results generated with radioactivedecay version 0.0.4.

In [1]:
import pickle
import radioactivedecay as rd
import numpy as np

benchmark_ver = '0.0.4'

print('Testing '+rd.__name__+' version '+rd.__version__+' with default dataset '+rd.DEFAULTDATA.dataset\
      +', benchmarking against version '+benchmark_ver+'.')

Testing radioactivedecay version 0.0.4 with default dataset icrp107, benchmarking against version 0.0.4.


Check radioactive decay calculations against baseline calculations. The validation strategy is check the relative difference of the results with the test version against the benchmark, and write out cases where the relative difference is greater than delta.

In [2]:
delta = 1E-14
hl_factors = [0.0, 1E-2, 1E-1, 1E0, 1E1, 1E2]

discrepancies = 0
for factor in hl_factors:
    benchmark = pickle.load(open('ver_'+benchmark_ver+'_halflife_'+str(factor)+'.pickle', 'rb'))
    for name in rd.DEFAULTDATA.nuclide_names:
        testversion = rd.Inventory({name: 1.0}).decay(rd.Radionuclide(name).halflife()*factor).contents
        denom = np.array(list(benchmark[name].values()), dtype=np.float64)
        rel_diff = np.abs(np.array(list(testversion.values()), dtype=np.float64)-denom)
        rel_diff = np.divide(rel_diff, denom, out=np.zeros_like(rel_diff), where=denom!=0)
        if np.max(rel_diff) > delta:
            print(factor, name, testversion, benchmark[name], rel_diff)
            discrepancies += 1


print('\nNumber of discrepancies with radioactivity relative delta > '+str(delta)+
      ' compared to benchmark case is '+str(discrepancies))


Number of discrepancies with radioactivity relative delta > 1e-14 compared to benchmark case is 0


Check radionuclide string, activity and time unit formatting.

In [3]:
print(rd.Inventory({'H3': 1}).contents)
print(rd.Inventory({'3H': 1.0}).contents)
print(rd.Inventory({'99mTc': 1E10}).contents)
print(rd.Inventory({'Tc99m': -1}).contents)
print(rd.Inventory({'152nEu': -1.0}).contents)
print(rd.Inventory({'Eu152n': -1E10}).contents)

print(rd.Inventory({'H3': 1}).decay(1, 'y').contents)
print(rd.Inventory({'99mTc': 1E10}).decay(1, 'd').contents)
print(rd.Inventory({'99mTc': 1E10}).decay(1, 'h').contents)
print(rd.Inventory({'152nEu': 1.0}).decay(1, 'm').contents)
print(rd.Inventory({'152nEu': 1.0}).decay(1, 's').contents)
print(rd.Inventory({'152nEu': 1.0}).decay(1).contents)

print(rd.Radionuclide('H3').halflife('y'))
print(rd.Radionuclide('H3').halflife('d')/365.2425)
print(rd.Radionuclide('99mTc').halflife('h'))
print(rd.Radionuclide('Tc99m').halflife('m')/60.0)
print(rd.Radionuclide('152nEu').halflife('m'))
print(rd.Radionuclide('Eu152n').halflife('s')/60.0)

{'H-3': 1}
{'H-3': 1.0}
{'Tc-99m': 10000000000.0}
{'Tc-99m': -1}
{'Eu-152n': -1.0}
{'Eu-152n': -10000000000.0}
{'H-3': 0.9452914876844037}
{'Tc-99': 30.458451846563033, 'Tc-99m': 629336340.3196114}
{'Tc-99': 3.53788983214965, 'Tc-99m': 8911554151.0408}
{'Eu-152': 9.70045914516703e-08, 'Eu-152n': 0.9928057204912689, 'Gd-152': 1.6532822233116719e-28, 'Nd-144': 0.0, 'Sm-148': 0.0}
{'Eu-152': 1.622489338586307e-09, 'Eu-152n': 0.9998796691881524, 'Gd-152': 4.60643985381057e-32, 'Nd-144': 0.0, 'Sm-148': 0.0}
{'Eu-152': 1.622489338586307e-09, 'Eu-152n': 0.9998796691881524, 'Gd-152': 4.60643985381057e-32, 'Nd-144': 0.0, 'Sm-148': 0.0}
12.32
12.319999999999999
6.015
6.015
96.0
96.0


Check Inventory returns correctly sorted inventories.

In [4]:
print(rd.Inventory({'U235': 1.0, 'H3': 1.0}).contents)
print(rd.Inventory({'U235': 1.0}).decay(0).contents)

{'H-3': 1.0, 'U-235': 1.0}
{'Ac-227': 0.0, 'At-219': 0.0, 'Bi-211': 0.0, 'Bi-215': 0.0, 'Fr-223': 0.0, 'Pa-231': 0.0, 'Pb-211': 0.0, 'Po-211': 0.0, 'Po-215': 0.0, 'Ra-223': 0.0, 'Rn-219': 0.0, 'Th-227': 0.0, 'Th-231': 0.0, 'Tl-207': 0.0, 'U-235': 1.0}
